In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim
import torch.autograd as autograd
import torchtext.vocab as torchvocab
from torch.autograd import Variable
import tqdm
import os
import time
import re
import pandas as pd
import string
import gensim
import time
import random
import collections
from collections import Counter
from nltk.corpus import stopwords
from itertools import chain
from sklearn.metrics import accuracy_score


In [4]:
wvmodel=gensim.models.Word2Vec.load("weibomodel").wv

In [5]:
data=pd.read_csv("yqdata.csv",usecols=['cut','fact','opinion','inhibition','fff',
                                      'activation','moral'])

In [6]:
x_data=data['cut'].values.tolist()

In [7]:
y_data=data['fact'].values.tolist()

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_data,
                                               y_data,test_size=0.20,random_state=1)

In [9]:
vocab = set(chain(*x_data))
vocab_size = len(vocab)

In [10]:
word_to_idx = {word: i+1 for i, word in enumerate(vocab)}
word_to_idx['<unk>'] = 0
idx_to_word = {i+1: word for i, word in enumerate(vocab)}
idx_to_word[0] = '<unk>'

In [11]:
def encode_samples(tokenized_samples):
    features = []
    for sample in tokenized_samples:
        feature = []
        for token in sample:
            if token in word_to_idx:
                feature.append(word_to_idx[token])
            else:
                feature.append(0)
        features.append(feature)
    return features

In [12]:
def pad_samples(features, maxlen, PAD=0):
    padded_features = []
    for feature in features:
        if len(feature) >= maxlen:
            padded_feature = feature[:maxlen]
        else:
            padded_feature = feature
            while(len(padded_feature) < maxlen):
                padded_feature.append(PAD)
        padded_features.append(padded_feature)
    return padded_features

In [13]:
#参数
MAX=20
num_epochs = 30
embed_size = 100
num_hiddens = 100
num_layers = 2
bidirectional = True
batch_size = 32
labels = 3
lr = 0.8
device = torch.device('cuda:0')
use_gpu = True

In [14]:
weight = torch.zeros(vocab_size+1, embed_size)
for i in range(len(wvmodel.index2word)):
    try:
        index = word_to_idx[wvmodel.index2word[i]]
    except:
        continue
    weight[index, :] = torch.from_numpy(wvmodel.get_vector(
        idx_to_word[word_to_idx[wvmodel.index2word[i]]]))

<ipython-input-14-ea485e14b06c>:7: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  weight[index, :] = torch.from_numpy(wvmodel.get_vector(


In [15]:
train_features = torch.tensor(pad_samples(encode_samples(x_train),maxlen=MAX))
train_labels = torch.tensor(y_train,dtype=torch.int64)
test_features = torch.tensor(pad_samples(encode_samples(x_test),maxlen=MAX))
test_labels = torch.tensor(y_test,dtype=torch.int64)

In [16]:
class SentimentNet(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 bidirectional, weight, labels, use_gpu, **kwargs):
        super(SentimentNet, self).__init__(**kwargs)
        self.num_hiddens = num_hiddens
        self.num_layers = num_layers
        self.use_gpu = use_gpu
        self.bidirectional = bidirectional
        self.embedding = nn.Embedding.from_pretrained(weight)
        self.embedding.weight.requires_grad = True
        self.encoder = nn.LSTM(input_size=embed_size, hidden_size=self.num_hiddens,
                               num_layers=num_layers, bidirectional=self.bidirectional,
                               dropout=0)
        if self.bidirectional:
            self.FC1 = nn.Linear(num_hiddens * 4, num_hiddens)
        else:
            self.FC1 = nn.Linear(num_hiddens * 2, num_hiddens)
        
        self.decoder=nn.Sequential(
            nn.Dropout(p=0.5),nn.ReLU(),nn.Linear(num_hiddens,labels))

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        states, hidden = self.encoder(embeddings.permute([1, 0, 2]))
        encoding = torch.cat([states[0], states[-1]], dim=1)
        tmp = self.FC1(encoding)
        outputs=self.decoder(tmp)
        return outputs

In [17]:
class textCNN(nn.Module):
    def __init__(self, vocab_size, embed_size, seq_len, labels, weight, **kwargs):
        super(textCNN, self).__init__(**kwargs)
        self.labels = labels
        self.embedding = nn.Embedding.from_pretrained(weight)
        self.embedding.weight.requires_grad = False
        self.conv1 = nn.Conv2d(1, 1, (2, embed_size))
        self.conv2 = nn.Conv2d(1, 1, (3, embed_size))
        self.conv3 = nn.Conv2d(1, 1, (4, embed_size))
        self.pool1 = nn.MaxPool2d((seq_len - 2 + 1, 1))
        self.pool2 = nn.MaxPool2d((seq_len - 3 + 1, 1))
        self.pool3 = nn.MaxPool2d((seq_len - 4 + 1, 1))
        self.linear = nn.Linear(3, labels)

    def forward(self, inputs):
        inputs = self.embedding(inputs).view(inputs.shape[0], 1, inputs.shape[1], -1)
        x1 = F.relu(self.conv1(inputs))
        x2 = F.relu(self.conv2(inputs))
        x3 = F.relu(self.conv3(inputs))

        x1 = self.pool1(x1)
        x2 = self.pool2(x2)
        x3 = self.pool3(x3)

        x = torch.cat((x1, x2, x3), -1)
        x = x.view(inputs.shape[0], 1, -1)

        x = self.linear(x)
        x = x.view(-1, self.labels)

        return(x)

In [18]:
#net = textCNN(vocab_size=(vocab_size+1), embed_size=embed_size,
#              seq_len=MAX, labels=labels, weight=weight)
#net.to(device)
#loss_function = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=lr)

In [19]:
net = SentimentNet(vocab_size=(vocab_size+1), embed_size=embed_size,
                   num_hiddens=num_hiddens, num_layers=num_layers,
                   bidirectional=bidirectional, weight=weight,
                   labels=labels, use_gpu=use_gpu)
net.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr)

In [20]:
train_set = torch.utils.data.TensorDataset(train_features, train_labels)
test_set = torch.utils.data.TensorDataset(test_features, test_labels)

In [21]:
train_iter = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                         shuffle=True)
test_iter = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                        shuffle=False)

In [22]:
for epoch in range(num_epochs):
    start = time.time()
    train_loss, test_losses = 0, 0
    train_acc, test_acc = 0, 0
    n, m = 0, 0
    for feature, label in train_iter:
        n += 1
#         net.train()
        net.zero_grad()
        feature = feature.to(device)
        label = label.to(device)
        score = net(feature)
        loss = loss_function(score, label)
        loss.backward()
#         scheduler.step()
        optimizer.step()
        train_acc += accuracy_score(torch.argmax(score.cpu().data,
                                                 dim=1), label.cpu())
        train_loss += loss
    with torch.no_grad():
        for test_feature, test_label in test_iter:
            m += 1
#             net.eval()
            test_feature = test_feature.cuda()
            test_label = test_label.cuda()
            test_score = net(test_feature)
            test_loss = loss_function(test_score, test_label)
            test_acc += accuracy_score(torch.argmax(test_score.cpu().data,
                                                    dim=1), test_label.cpu())
            test_losses += test_loss
    
    end = time.time()
    runtime = end - start
    print('epoch: %d, train loss: %.4f, train acc: %.4f, test loss: %.4f, test acc: %.4f, time: %.2f' %
          (epoch, train_loss.data / n, train_acc / n, test_losses.data / m, test_acc / m, runtime))
    


epoch: 0, train loss: 0.8324, train acc: 0.6156, test loss: 0.8622, test acc: 0.6072, time: 1.10
epoch: 1, train loss: 0.8171, train acc: 0.6296, test loss: 0.8638, test acc: 0.6072, time: 0.80
epoch: 2, train loss: 0.8185, train acc: 0.6263, test loss: 0.8679, test acc: 0.6072, time: 0.79
epoch: 3, train loss: 0.8169, train acc: 0.6288, test loss: 0.8581, test acc: 0.6072, time: 0.77
epoch: 4, train loss: 0.8155, train acc: 0.6304, test loss: 0.8597, test acc: 0.6072, time: 0.76
epoch: 5, train loss: 0.8165, train acc: 0.6284, test loss: 0.8518, test acc: 0.6072, time: 0.79
epoch: 6, train loss: 0.8146, train acc: 0.6293, test loss: 0.8789, test acc: 0.6072, time: 0.76
epoch: 7, train loss: 0.8104, train acc: 0.6293, test loss: 0.8633, test acc: 0.6072, time: 0.76
epoch: 8, train loss: 0.8089, train acc: 0.6308, test loss: 0.8459, test acc: 0.6090, time: 0.79
epoch: 9, train loss: 0.8099, train acc: 0.6243, test loss: 0.8451, test acc: 0.6042, time: 0.76
epoch: 10, train loss: 0.8037,